In [121]:
import pandas as pd, numpy as np
import itertools, requests, json

In [122]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')
lm=pd.read_excel('data/szekelyhon_locs_curated.xlsx')

In [123]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [124]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]
lm["uid"] = lm["title"].map(str) + lm["content"]

In [125]:
dc=dc.set_index('uid')[['relevant','severity','deaths','duplicate']]
dm=dm.set_index('uid')
lm=lm.set_index('uid')[['predicted_locs','actual_locs']]

In [126]:
dmi=dm.loc[dc.index.dropna()]

In [127]:
df=dmi.join(dc).drop_duplicates()

In [128]:
dl=df.join(lm)

In [129]:
len(dl)

7

Working df ready. Now infer location.

In [130]:
to_geo=[i.split(',') for i in dl['actual_locs'].unique() if str(i)!='nan']

In [131]:
to_geo=[i.strip() for i in list(itertools.chain.from_iterable(to_geo))]

Load HU names

In [132]:
coords=json.loads(open('geo/huncoords.json','r').read())
megyek=json.loads(open('geo/megyek.json','r').read())

In [133]:
data=[]
header=['index']
for i in dl.iteritems():
    header.append(i[0])
header.append('geo_loc')
header.append('latitude')
header.append('longitude')
header.append('id')
for i in dl.T.iteritems():
    dummy=[i[0]]
    for j in range(len(i[1])):
        if j==(len(i[1])-1):
            dummy.append(i[1][j])
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dtime=str(dummy[1])[:10]
            if dtime=='nan': dtime=u'N/A' 
            if str(i[1][j])!='nan':    
                for k in i[1][j].split(','):
                    tk=k.strip().replace('[','').replace(']','').replace("'",'')
                    dummy[-4]=tk
                    if tk in coords:
                        dummy[-3]=coords[tk][0]
                        dummy[-2]=coords[tk][1]
                    else:
                        dummy[-3]=''
                        dummy[-2]=''
                    #dummy[-1]=str(dummy[-3])+' ('+str(dummy[0]+1)+')\n'+dtime
                    dummy[-1]=str(dummy[-4])+', '+dtime
                    data.append(list(dummy))                    
            else:
                dummy[-1]=u'N/A' +', '+dtime
                data.append(list(dummy))
        else: 
            dummy.append(i[1][j])

In [134]:
df=pd.DataFrame(data)
df.columns=header
df

,index,Unnamed: 0,date,hudate,link,image,category,icategory,tags,title,...,relevant,severity,deaths,duplicate,predicted_locs,actual_locs,geo_loc,latitude,longitude,id
0,Medve Szentegyháza határábanNem kis fennakadás...,1,2019-06-07,2019. június 07.,https://szekelyhon.ro/aktualis/medve-szentegyh...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['medve', 'Szentegyháza']",Medve Szentegyháza határában,...,1,1,0,0,Szentegyháza,Szentegyháza,Szentegyháza,46.3267,25.5148,"Szentegyháza, 1"
1,Amikor farkas provokálja a medvéketRendhagyó v...,3,2019-06-13,2019. június 13.,https://szekelyhon.ro/aktualis/amikor-farkas-p...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Háromszék', 'medve']",Amikor farkas provokálja a medvéket,...,1,1,0,0,Sepsibükszád,Sepsibükszád,Sepsibükszád,46.0077,25.8411,"Sepsibükszád, 3"
2,"Medvebocsot kínoztak, eljárás indul ellenükNag...",4,2019-06-17,2019. június 17.,https://szekelyhon.ro/aktualis/medvebocsot-kin...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['Brassó megye', 'medve']","Medvebocsot kínoztak, eljárás indul ellenük",...,1,1,0,0,"['Brassó', 'Ágostonfalva', 'Brassó', 'Ágostonf...",Ágostonfalva,Ágostonfalva,46.0046,25.5553,"Ágostonfalva, 4"
3,Medvecsalád bukkant fel Csíkszentimre határába...,6,2019-06-25,2019. június 25.,https://szekelyhon.ro/aktualis/medvecsalad-buk...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Csíkszentimre', 'medve']",Medvecsalád bukkant fel Csíkszentimre határában,...,1,1,0,0,Csíkszentimre,Csíkszentimre,Csíkszentimre,46.2617,25.7966,"Csíkszentimre, 6"
4,"Semmi különös, csak a székely eteti a medvét a...",7,2019-06-28,2019. június 28.,https://szekelyhon.ro/aktualis/semmi-kulonos-c...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Háromszék', 'medve']","Semmi különös, csak a székely eteti a medvét a...",...,1,1,0,0,"['Ozsdola', 'Újszékely', 'Ozsdola']",Ozsdola,Ozsdola,45.9805,26.263,"Ozsdola, 7"
5,Túlélte a gázolást a Brassó határában elütött ...,8,2019-06-28,2019. június 28.,https://szekelyhon.ro/aktualis/tulelte-a-gazol...,https://media.szekelyhon.ro/pictures/123rf/201...,aktualis,123rf,"['Brassó', 'medve']",Túlélte a gázolást a Brassó határában elütött ...,...,1,0,0,0,Brassó,Brassó,Brassó,45.658,25.6012,"Brassó, 8"
6,"Íme, a vitatéma: megszámolták a medvéketÉvről ...",9,NaT,NaN,https://szekelyhon.ro/aktualis/ime-a-vitatema-...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['Hargita megye', 'Maros megye', 'medve']","Íme, a vitatéma: megszámolták a medvéket",...,1,0,0,0,NaN,NaN,,,,"N/A, 9"


Save data for loading in viz app

In [135]:
df['checked']=''
df['exact_loc']=0
df['source']='Székelyhon'

In [136]:
df=df[['checked',
    'date',
    'link',
    'image',
    'source',
    'title',
    'content',
    'relevant',
    'severity',
    'deaths',
    'duplicate',
    'geo_loc',
    'exact_loc',
    'latitude',
    'longitude']].set_index('checked')

In [137]:
#PowerBI exploration data
df.to_excel('data/szekelyhon_data.xlsx')